In [1]:
import pandas as pd
import os

sma_df = pd.read_excel('/Users/danieljoo/Library/CloudStorage/OneDrive-YaleUniversity/mda_movr/new analysis/sma_data/sma_clean.xlsx')

/Users/danieljoo/miniconda3/envs/dj/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def read_excel_files(folder_path):
    excel_dict = {}
    
    # Get a list of all files in the specified folder
    files = os.listdir(folder_path)
    
    # Iterate over each file in the folder
    for file in files:
        # Check if the file has an Excel extension (.xlsx or .xls)
        if file.endswith('.xlsx') or file.endswith('.xls'):
            file_path = os.path.join(folder_path, file)
            
            # Read the Excel file into a DataFrame
            df = pd.read_excel(file_path)
            
            # Add the DataFrame to the dictionary with the file name as the key
            excel_dict[file] = df
    
    return excel_dict

excel_dict = read_excel_files('./sma_data')

print("Sheets in the Excel dict:")
for sheet_name in excel_dict.keys():
    print(sheet_name)
def remove_high_missing_columns(df, threshold=0.99):
    missing_percentages = df.isnull().mean()
    
    # Identify columns with missing values above the threshold
    columns_to_remove = missing_percentages[missing_percentages > threshold].index
    
    # Remove the identified columns from the DataFrame
    df = df.drop(columns=columns_to_remove)
    return df


demographics_df = remove_high_missing_columns(excel_dict['DEMO.xlsx'])
demographics_df = demographics_df.drop(columns=['hltinosp', 'nonmdapc', 'inschool', 'inschyes', 'edulvl', 'edulvl1', 'edulvl2', 'employ', 'has_mname', 'dob1'])
enco_df = remove_high_missing_columns(excel_dict['ENCO.xlsx'])
hospitalizations_df = remove_high_missing_columns(excel_dict['Hospitalization Repeat Group.xlsx'])
medications_df = remove_high_missing_columns(excel_dict['Medication Repeat Group.xlsx'])
diagnosis_df = remove_high_missing_columns(excel_dict['DIAG.xlsx'])
discontinuation_df = remove_high_missing_columns(excel_dict['DISC.xlsx'])
pulmonary_df = remove_high_missing_columns(excel_dict['Pulmonary Device Repeat Group.xlsx'])
dob_df = pd.read_excel('./sma_data/SMA-DOB_May2024.xlsx')
demographics_df = demographics_df.merge(dob_df, on='FACPATID', how='left')

Sheets in the Excel dict:
Spinraza Maintenance Dose Repeat Group.xlsx
SMA-DOB_May2024.xlsx
NPI Repeat Group.xlsx
Pulmonary Device Repeat Group.xlsx
Trial Details.xlsx
Surgery Repeat Group.xlsx
sma_clean.xlsx
Fundoplication Nissen Repeat Group.xlsx
Assisstive Device Repeat Group.xlsx
Hospitalization Repeat Group.xlsx
DISC.xlsx
LOG.xlsx
DIAG.xlsx
ENCO.xlsx
Medication Repeat Group.xlsx
DEMO.xlsx


In [3]:
sma_df = pd.merge(sma_df, demographics_df[['FACPATID', 'hltin']], on='FACPATID', how='left')

In [4]:
demographics_df

,FACPATID,CASE_ID,SCHEDULED_FORM_NAME,FORM_STATUS,Dataset,age,dstype,enroldt,gender,ethnic,ethnicosp,ethnicity,hltin,sex,enroldt.P,dob.P,dob1.P,dob
0,1001-1,1,Demographics,Complete,MOVR,26,SMA,2022-11-14,Male,White,NaN,Not Hispanic or Latino,Medicaid,M,day,day,day,1997-04-11
1,1001-5,5,Demographics,Complete,MOVR,4,SMA,2022-11-16,Male,White,NaN,Not Hispanic or Latino,Private or group health insurance,NaN,day,day,NaN,2019-09-13
2,1001-6,6,Demographics,Complete,MOVR,4,SMA,2022-11-16,Male,White,NaN,Not Hispanic or Latino,Private or group health insurance,NaN,day,day,NaN,2019-09-13
3,1048-29,29,Demographics,Complete,MOVR,35,SMA,2022-05-25,Female,White,NaN,Not Hispanic or Latino,Private or group health insurance,F,day,day,day,1988-09-14
4,1048-30,30,Demographics,Complete,MOVR,36,SMA,2022-05-25,Male,Unknown,NaN,Unknown,Private or group health insurance,M,day,day,day,1987-06-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,1449-55,55,Demographics,Complete,MOVR,2,SMA,2022-09-21,Female,White,NaN,Not Hispanic or Latino,Private or group health insurance,NaN,day,day,NaN,2021-06-06
351,1449-56,56,Demographics,Complete,MOVR,2,SMA,2022-09-21,Female,White,NaN,Not Hispanic or Latino,Private or group health insurance,NaN,day,day,NaN,2021-04-14
352,9000-2,2,Demographics,Complete,MOVR,56,SMA,2020-03-04,Female,White,NaN,Not Reported,Private or group health insurance,F,day,day,day,1968-01-03
353,9001-49,49,Demographics,Complete,MOVR,5,SMA,2020-08-26,Male,Black or African American,NaN,Not Hispanic or Latino,Medicaid,NaN,day,day,NaN,2018-06-18


we reinterpret the health insurance as follows

1. if a patient has private or group health insurance, he is labelled as 'has private or group health insurance'
2. if a patient is solely depenedent on medicaid, 'medicaid only'
3. if a pediatric patient has both medicare and medicaid, it is interpreted as (2). likely a logging error
4. Other situations, like VA benefits, dual eligible patients, no insurance, or unknown insurance are grouped into 'other' and are excluded from analysis.

In [5]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(sma_df):
    def simplify_hltin(row):
        insurance = row['hltin']
        age = row['age_dx']
        # Handle NaN values
        if pd.isna(insurance):
            return insurance
        insurance = str(insurance).lower()
        # Special case for pediatric patients with Medicare+Medicaid
        if age < 65 and insurance == 'medicaid,medicare':
            return 'only medicaid'
        # Original logic
        if 'private or group health insurance' in insurance:
            return 'has private or group health insurance'
        elif insurance == 'medicaid':
            return 'only medicaid'
        else:
            return 'other'
    sma_df['new_hltin'] = sma_df.apply(simplify_hltin, axis=1)
    sma_df = sma_df[['FACPATID', 'age_dx', 'Sheet2.gender', 'new_hltin', 'meds', 'Sheet2.smaclass', 'Sheet1.ethnic']]
    sma_df.rename(columns={'Sheet2.gender':'gender', 'new_hltin':'hltin', 'Sheet2.smaclass':'sma_type', 'Sheet1.ethnic': 'ethnic'}, inplace=True)
    return sma_df

sma_df_clean = clean_data(sma_df.copy())
sma_df_clean.head()

,FACPATID,age_dx,gender,hltin,meds,sma_type,ethnic
0,1001-1,1.726027,Male,only medicaid,NaN,2,White
1,1048-29,3.298630,Female,has private or group health insurance,Spinraza Only,3,White
2,1048-5,2.345205,Male,other,NaN,2,Black or African American
3,1063-8,0.246575,Male,has private or group health insurance,NaN,1,Black or African American
4,1064-2,0.641096,Male,only medicaid,Spinraza Only,2,White


In [6]:
df = sma_df_clean.copy()
df

,FACPATID,age_dx,gender,hltin,meds,sma_type,ethnic
0,1001-1,1.726027,Male,only medicaid,NaN,2,White
1,1048-29,3.298630,Female,has private or group health insurance,Spinraza Only,3,White
2,1048-5,2.345205,Male,other,NaN,2,Black or African American
3,1063-8,0.246575,Male,has private or group health insurance,NaN,1,Black or African American
4,1064-2,0.641096,Male,only medicaid,Spinraza Only,2,White
...,...,...,...,...,...,...,...
254,1449-50,0.021918,Male,has private or group health insurance,NaN,1,Other
255,1449-53,0.027397,Male,has private or group health insurance,Risdiplam Only,Pre-symptomatic,White
256,1449-55,0.027397,Female,has private or group health insurance,NaN,1,White
257,1449-56,0.032877,Female,has private or group health insurance,NaN,1,White


In [7]:

df = df[df['meds'] != 'Combination']
df = df[df['age_dx'] < 2]
df = df[df['sma_type']!='3']
df = df[df['hltin']!= 'other']

df = pd.merge(df, demographics_df[['FACPATID', 'dob']], on='FACPATID', how='inner')

df['dob'] = pd.to_datetime(df['dob'])
df = df[df['dob'] > '2017-05-24']

df = df[df['hltin'] != 'only medicaid']
df = df[df['meds'] == 'Zolgensma Only']
df

,FACPATID,age_dx,gender,hltin,meds,sma_type,ethnic,dob
20,1095-62,0.016438,Male,has private or group health insurance,Zolgensma Only,Pre-symptomatic,Other,2022-02-04
25,1225-19,1.556164,Female,has private or group health insurance,Zolgensma Only,2,White,2019-04-17
30,1225-5,0.013699,Male,has private or group health insurance,Zolgensma Only,Pre-symptomatic,White,2020-06-17
39,1259-14,1.106849,Male,has private or group health insurance,Zolgensma Only,2,White,2018-06-16
41,1259-16,0.063014,Male,has private or group health insurance,Zolgensma Only,1,White,2020-12-29
45,1259-25,0.268493,Male,has private or group health insurance,Zolgensma Only,1,White,2020-06-26
62,1405-20,0.276712,Male,has private or group health insurance,Zolgensma Only,1,Other,2021-03-05
101,1433-18,0.084932,Male,has private or group health insurance,Zolgensma Only,Pre-symptomatic,White,2021-02-11
104,1433-23,0.021918,Female,has private or group health insurance,Zolgensma Only,Pre-symptomatic,White,2021-11-10
106,1433-29,0.035616,Female,has private or group health insurance,Zolgensma Only,1,White,2019-11-14


In [8]:
risdiplam_df = sma_df_clean.copy()
risdiplam_df = pd.merge(risdiplam_df, demographics_df[['FACPATID', 'dob']], on='FACPATID', how='inner')
risdiplam_df = risdiplam_df[risdiplam_df['hltin'] != 'other']



risdiplam_df = risdiplam_df[risdiplam_df['meds']!= 'Combination']

risdiplam_df = risdiplam_df[risdiplam_df['meds'] == 'Spinraza Only']
risdiplam_df = risdiplam_df[risdiplam_df['hltin'] != 'only medicaid']
risdiplam_df

,FACPATID,age_dx,gender,hltin,meds,sma_type,ethnic,dob
1,1048-29,3.298630,Female,has private or group health insurance,Spinraza Only,3,White,1988-09-14
6,1064-28,9.789041,Male,has private or group health insurance,Spinraza Only,3,White,1986-03-21
16,1095-22,3.958904,Female,has private or group health insurance,Spinraza Only,3,White,1990-01-17
25,1095-40,0.076712,Male,has private or group health insurance,Spinraza Only,Pre-symptomatic,White,2021-03-03
27,1095-44,3.482192,Male,has private or group health insurance,Spinraza Only,3,White,2009-07-10
31,1095-48,0.657534,Male,has private or group health insurance,Spinraza Only,2,Other,2002-12-04
37,1095-56,0.021918,Male,has private or group health insurance,Spinraza Only,1,White,2021-11-09
50,1184-141,3.978082,Female,has private or group health insurance,Spinraza Only,3,Black or African American,1976-08-10
57,1225-17,1.153425,Male,has private or group health insurance,Spinraza Only,2,White,2004-01-05
61,1225-24,3.353425,Male,has private or group health insurance,Spinraza Only,3,White,2014-03-26


In [9]:
import pandas as pd
import numpy as np

def calculate_percentage(count, total):
    """Calculate percentage safely avoiding division by zero"""
    if total == 0:
        return "0 (0.0%)"  # Added % symbol
    return f"{count} ({(count/total*100):.1f}%)"  # Added % symbol

def create_clinical_table(sma_df_clean, include_percentages=True):
    """
    Create a comprehensive clinical characteristics table for SMA patients by treatment type
    """
    # Define medication groups in specified order
    treatments = ['None', 'Risdiplam Only', 'Spinraza Only', 'Zolgensma Only', 'Combination']
    
    # Initialize dictionary to store results
    results = {}
    
    for treatment in treatments:
        # Special handling for 'None' category
        if treatment == 'None':
            group = sma_df_clean[sma_df_clean['meds'].isna()]
        else:
            group = sma_df_clean[sma_df_clean['meds'] == treatment]
            
        n = len(group)
        
        # Create column header with n
        col_name = f"{treatment} (n={n})"
        results[col_name] = {}
        
        # Age at diagnosis
        if n > 0:
            mean_age = group['age_dx'].mean()
            std_age = group['age_dx'].std()
            results[col_name]['Age at diagnosis, mean (sd)'] = f"{mean_age:.2f} ({std_age:.2f})"
        else:
            results[col_name]['Age at diagnosis, mean (sd)'] = "NA"
        
        # For counts-only version
        if not include_percentages:
            # SMA Class
            results[col_name]['SMA Class'] = ""
            results[col_name]['Pre-symptomatic'] = str((group['sma_type'] == 'Pre-symptomatic').sum())
            results[col_name]['1'] = str((group['sma_type'] == '1').sum())
            results[col_name]['2'] = str((group['sma_type'] == '2').sum())
            results[col_name]['3'] = str((group['sma_type'] == '3').sum())
            
            # Race
            results[col_name]['Race'] = ""
            results[col_name]['Black or African American'] = str((group['ethnic'] == 'Black or African American').sum())
            results[col_name]['White'] = str((group['ethnic'] == 'White').sum())
            results[col_name]['Other'] = str((~group['ethnic'].isin(['Black or African American', 'White']) & group['ethnic'].notna()).sum())
            
            # Sex
            results[col_name]['Sex'] = ""
            results[col_name]['Male'] = str((group['gender'] == 'Male').sum())
            results[col_name]['Female'] = str((group['gender'] == 'Female').sum())
            
            # Insurance Status
            results[col_name]['Insurance Status'] = ""
            results[col_name]['Medicaid Only'] = str((group['hltin'] == 'only medicaid').sum())
            results[col_name]['Private/Group'] = str((group['hltin'] == 'has private or group health insurance').sum())
            results[col_name]['Other Insurance'] = str((~group['hltin'].isin(['only medicaid', 'has private or group health insurance']) & group['hltin'].notna()).sum())
        
        else:
            # SMA Class
            results[col_name]['SMA Class'] = ""
            count_presymp = (group['sma_type'] == 'Pre-symptomatic').sum()
            count_1 = (group['sma_type'] == '1').sum()
            count_2 = (group['sma_type'] == '2').sum()
            count_3 = (group['sma_type'] == '3').sum()
            
            results[col_name]['Pre-symptomatic'] = calculate_percentage(count_presymp, n)
            results[col_name]['1'] = calculate_percentage(count_1, n)
            results[col_name]['2'] = calculate_percentage(count_2, n)
            results[col_name]['3'] = calculate_percentage(count_3, n)
            
            # Race
            results[col_name]['Race'] = ""
            count_aa = (group['ethnic'] == 'Black or African American').sum()
            count_white = (group['ethnic'] == 'White').sum()
            count_other = (~group['ethnic'].isin(['Black or African American', 'White']) & group['ethnic'].notna()).sum()
            
            results[col_name]['African American'] = calculate_percentage(count_aa, n)
            results[col_name]['White'] = calculate_percentage(count_white, n)
            results[col_name]['Other'] = calculate_percentage(count_other, n)
            
            # Sex
            results[col_name]['Sex'] = ""
            count_male = (group['gender'] == 'Male').sum()
            count_female = (group['gender'] == 'Female').sum()
            
            results[col_name]['Male'] = calculate_percentage(count_male, n)
            results[col_name]['Female'] = calculate_percentage(count_female, n)
            
            # Insurance Status
            results[col_name]['Insurance Status'] = ""
            count_medicaid = (group['hltin'] == 'only medicaid').sum()
            count_private = (group['hltin'] == 'has private or group health insurance').sum()
            count_other_ins = (~group['hltin'].isin(['only medicaid', 'has private or group health insurance']) & group['hltin'].notna()).sum()
            
            results[col_name]['Medicaid Only'] = calculate_percentage(count_medicaid, n)
            results[col_name]['Private/Group'] = calculate_percentage(count_private, n)
            results[col_name]['Other Insurance'] = calculate_percentage(count_other_ins, n)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

# Rest of the code remains the same...

def style_clinical_table(df):
    """
    Apply styling to the clinical characteristics table
    """
    # Define header rows (rows that shouldn't have numbers)
    header_rows = ['SMA Class', 'Race', 'Sex', 'Insurance Status']
    
    def style_row(row):
        if row.name in header_rows:
            return ['font-weight: bold'] * len(row)
        return [''] * len(row)
    
    return df.style\
        .apply(style_row, axis=1)\
        .set_properties(**{
            'text-align': 'center',
            'padding': '5px'
        })\
        .set_table_styles([
            {'selector': 'th',
             'props': [('text-align', 'left'),
                      ('font-weight', 'bold'),
                      ('padding', '5px')]},
        ])

# Usage example for table with percentages:
# summary_table_with_pct = create_clinical_table(sma_df_clean, include_percentages=True)
# styled_table_with_pct = style_clinical_table(summary_table_with_pct)
# display(styled_table_with_pct)

# Usage example for table without percentages:
# summary_table_no_pct = create_clinical_table(sma_df_clean, include_percentages=False)
# styled_table_no_pct = style_clinical_table(summary_table_no_pct)
# display(styled_table_no_pct)

In [10]:
# Create and display table with percentages
summary_table_with_pct = create_clinical_table(sma_df_clean, include_percentages=True)
styled_table_with_pct = style_clinical_table(summary_table_with_pct)
display(styled_table_with_pct)

# Create and display table without percentages
summary_table_no_pct = create_clinical_table(sma_df_clean, include_percentages=False)
styled_table_no_pct = style_clinical_table(summary_table_no_pct)
display(styled_table_no_pct)

,None (n=117),Risdiplam Only (n=10),Spinraza Only (n=89),Zolgensma Only (n=18),Combination (n=25)
"Age at diagnosis, mean (sd)",3.78 (8.84),1.79 (2.23),3.95 (7.98),0.32 (0.54),0.49 (0.60)
SMA Class,,,,,
Pre-symptomatic,11 (9.4%),2 (20.0%),3 (3.4%),9 (50.0%),3 (12.0%)
1,44 (37.6%),4 (40.0%),30 (33.7%),6 (33.3%),15 (60.0%)
2,36 (30.8%),3 (30.0%),28 (31.5%),3 (16.7%),5 (20.0%)
3,26 (22.2%),1 (10.0%),28 (31.5%),0 (0.0%),2 (8.0%)
Race,,,,,
African American,11 (9.4%),2 (20.0%),9 (10.1%),3 (16.7%),2 (8.0%)
White,78 (66.7%),7 (70.0%),67 (75.3%),13 (72.2%),18 (72.0%)
Other,28 (23.9%),1 (10.0%),13 (14.6%),2 (11.1%),5 (20.0%)


,None (n=117),Risdiplam Only (n=10),Spinraza Only (n=89),Zolgensma Only (n=18),Combination (n=25)
"Age at diagnosis, mean (sd)",3.78 (8.84),1.79 (2.23),3.95 (7.98),0.32 (0.54),0.49 (0.60)
SMA Class,,,,,
Pre-symptomatic,11,2,3,9,3
1,44,4,30,6,15
2,36,3,28,3,5
3,26,1,28,0,2
Race,,,,,
Black or African American,11,2,9,3,2
White,78,7,67,13,18
Other,28,1,13,2,5


In [11]:
import numpy as np
from scipy.stats import fisher_exact
import pandas as pd

# Creating the contingency table
data = {
    'Medicaid Only': [54, 3, 23, 4, 9],
    'Private/Group': [44, 6, 40, 14, 11]
}

# Convert to DataFrame for better visualization
df = pd.DataFrame(data, index=['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5'])
print("Contingency Table:")
print(df)
print("\nFisher's Exact Test Results:")

# Perform Fisher's exact test
oddsratio, pvalue = fisher_exact(df)

print(f"Odds Ratio: {oddsratio:.4f}")
print(f"P-value: {pvalue:.4f}")

# Calculate row and column totals
df['Row Total'] = df.sum(axis=1)
col_totals = df.sum()
df.loc['Column Total'] = col_totals

print("\nContingency Table with Totals:")
print(df)

# Calculate expected frequencies
row_totals = df['Row Total'][:-1]  # Exclude the total row
col_totals = df.iloc[:-1, :2].sum()  # Exclude the total column
grand_total = row_totals.sum()

print("\nExpected Frequencies:")
expected = pd.DataFrame(index=df.index[:-1], columns=df.columns[:2])
for i in df.index[:-1]:
    for j in df.columns[:2]:
        expected.loc[i, j] = (row_totals[i] * col_totals[j]) / grand_total

print(expected.round(2))

Contingency Table:
            Medicaid Only  Private/Group
Category 1             54             44
Category 2              3              6
Category 3             23             40
Category 4              4             14
Category 5              9             11

Fisher's Exact Test Results:


ValueError: The input `table` must be of shape (2, 2).

In [ ]:
summary_table_no_pct.to_excel('summary_table_no_pct.xlsx', index=True)